In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Test set manipulation

In [2]:
test= pd.read_csv('data/test.csv', parse_dates = True)

/opt/anaconda3/envs/minicomp/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
test.loc[:,'Date'] = pd.to_datetime(test.loc[:,'Date'], format='%Y-%m-%d')
test.loc[:,'DayOfWeek'] = test.loc[:,'Date'].dt.weekday + 1

test.loc[test['StateHoliday'] == 0.0, 'StateHoliday'] = '0'
test.loc[test['StateHoliday'] == 0, 'StateHoliday'] = '0'

In [4]:
test.drop(['Customers', 'SchoolHoliday'], axis = 1, inplace = True)

In [5]:
test['Month'] = test['Date'].dt.month

In [6]:
test.loc[test['Month'] != 12, 'Month'] = 0
test.loc[test['Month'] == 12, 'Month'] = 1

In [7]:
test

,Date,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,Month
0,2014-08-01,371,5,7195,1,1,0,0
1,2014-08-01,372,5,9525,1,1,0,0
2,2014-08-01,373,5,5621,1,1,0,0
3,2014-08-01,380,5,16933,1,1,0,0
4,2014-08-01,374,5,7828,1,1,0,0
...,...,...,...,...,...,...,...,...
379430,2015-07-31,746,5,9082,1,1,0,0
379431,2015-07-31,747,5,10708,1,1,0,0
379432,2015-07-31,748,5,7481,1,1,0,0
379433,2015-07-31,742,5,10460,1,1,0,0


In [5]:
stores_light = pd.read_csv('stores_light.csv')
stores_light.loc[878, 'StoreType'] = d
stores_light.loc[878, 'Assortment'] = a
stores_light.loc[878, 'CompetitionDistance'] = 2500
test = test.merge(stores_light, how = 'left', on = 'Store')

NameError: name 'test' is not defined

In [9]:
test

,Date,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,Month,StoreType,Assortment,CompetitionDistance
0,2014-08-01,371,5,7195,1,1,0,0,d,c,1970.0
1,2014-08-01,372,5,9525,1,1,0,0,d,c,4880.0
2,2014-08-01,373,5,5621,1,1,0,0,d,c,11120.0
3,2014-08-01,380,5,16933,1,1,0,0,a,a,2240.0
4,2014-08-01,374,5,7828,1,1,0,0,a,a,1150.0
...,...,...,...,...,...,...,...,...,...,...,...
379430,2015-07-31,746,5,9082,1,1,0,0,d,c,4330.0
379431,2015-07-31,747,5,10708,1,1,0,0,c,c,45740.0
379432,2015-07-31,748,5,7481,1,1,0,0,d,a,2380.0
379433,2015-07-31,742,5,10460,1,1,0,0,d,c,4380.0


In [10]:
train = pd.read_csv('data/train.csv', parse_dates=True)

train.loc[(train.loc[:, 'Sales'].isnull()) & (train['Open'] == 0), 'Sales'] = 0
train.loc[(train.loc[:, 'Sales'].isnull()) & (train['Customers'] == 0), 'Sales'] = 0

train.loc[(train.loc[:, 'Customers'].isnull()) & (train['Sales'] == 0), 'Customers'] = 0

no_null_sales = train.loc[train['Sales'].notnull(), :]
no_null_sales = no_null_sales[train['Open'] != 0]

no_null_sales = no_null_sales.loc[no_null_sales['Store'].notnull(), :]

no_null_sales.loc[:,'Date'] = pd.to_datetime(no_null_sales.loc[:,'Date'], format='%Y-%m-%d')
no_null_sales.loc[:,'DayOfWeek'] = no_null_sales.loc[:,'Date'].dt.weekday + 1

no_null_sales.loc[no_null_sales['StateHoliday'] == 0.0, 'StateHoliday'] = '0'
no_null_sales.loc[no_null_sales['StateHoliday'] == 0, 'StateHoliday'] = '0'

no_null_sales.loc[(no_null_sales['Open'].isnull()) & (no_null_sales['Sales'] == 0),'Open'] = 0
no_null_sales.loc[(no_null_sales['Open'].isnull()) & (no_null_sales['Sales'] > 0),'Open'] = 1

no_closed_stores = no_null_sales[no_null_sales['Open'] == 1]
no_closed_stores = no_closed_stores.merge(stores_light, how = 'left', on = 'Store')

no_closed_stores = no_closed_stores[no_closed_stores['Sales'] > 800]

/opt/anaconda3/envs/minicomp/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<ipython-input-10-212a77686b9d>:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  no_null_sales = no_null_sales[train['Open'] != 0]


In [11]:
assortment_dict = no_closed_stores.groupby('Assortment').mean()['Sales'].to_dict()
storetype_dict = no_closed_stores.groupby('StoreType').mean()['Sales'].to_dict()
store_dict = no_closed_stores.groupby('Store').mean()['Sales'].to_dict()
holiday_dict = no_closed_stores.groupby('StateHoliday').mean()['Sales'].to_dict()

In [12]:
test['Store'] = test['Store'].map(store_dict)
test['StateHoliday'] = test['StateHoliday'].map(holiday_dict)

test['StoreType'] = test['StoreType'].map(storetype_dict)
test['Assortment'] = test['Assortment'].map(assortment_dict)

test['PromoMonday'] = 0
test.loc[(test['Promo'] == 1) & (test['DayOfWeek'] == 1), 'PromoMonday'] = 1

## Download, import model and make predictions

In [1]:
url = "https://www.dropbox.com/s/q2e41mzg759fzzu/model.sav?dl=1"  # dl=1 is important
import urllib.request
u = urllib.request.urlopen(url)
data = u.read()
u.close()
 
with open('model.sav', "wb") as f :
    f.write(data)

In [13]:
import pickle
model = pickle.load(open('model.sav', 'rb'))

In [14]:
preds = model.predict(test.drop(['Date', 'Sales'], axis = 1))
actuals = test['Sales']

## Get score

In [15]:
def metric(preds, actuals):
    preds = preds.reshape(-1)
    actuals = actuals.reshape(-1)
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])

def get_score(actuals, preds):
    new_test= pd.DataFrame({'Actuals': actuals,'Preds': preds})
    new_test = new_test.loc[new_test['Actuals'] != 0,:]
    return metric(np.array(new_test['Actuals']), np.array(new_test['Preds']))

In [16]:
get_score(actuals, preds)

19.046137383492127